In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ase.db
import wget
import pandas as pd
import c2dbtools as c2t

ModuleNotFoundError: No module named 'c2dbtools'

In [9]:
# Connect to database
db = ase.db.connect('c2db.db')

options = 'is_magnetic=True, thermodynamic_stability_level=3'
rows = db.select(options)

props = ["formula" ,"spgnum", "spacegroup","dE_zx", "dE_zy", "lam","magmom", "magstate", "doi","uid"] 
raw_df = pd.DataFrame( [ [ x.get(p) for p in props ] for x in rows ] , columns = props);

import json
for i,(formula, db_spgn,db_spg,uid) in enumerate(zip(raw_df.formula,raw_df.spgnum, raw_df.spacegroup,raw_df.uid)):
    if formula=="CrI2":
        syst_data= c2t.get_online_data(uid , verbose=False);
        try:
            cell    = c2t.get_cell(syst_data);
            symmetry= c2t.get_symmetry(cell);
            spg = symmetry["international"]; spgn = symmetry["number"];
            ptg = symmetry["pointgroup"];
            match = spgn==db_spgn;
            if not match:
                print( i,formula," database = {} ({}) computed = {} ({}), match={}".format(db_spg,db_spgn,spg,spgn,match ))
            else:
                print( formula," {} ({}) [{}]".format(spg,spgn,ptg ))                
                sym_ops= c2t.get_symmetry_matrices(cell);
                Rs, ts = sym_ops['rotations'] , sym_ops['translations'];
                for v, k in c2t.symmetry_directions(cell).items():
                    print(v,k)
                for R in Rs:
                    print( c2t.sym_label(R,cell) );
        except:
            print(i,"data of",uid," not complete",)

CrI2  P-4m2 (115) [-42m]
v1 [ 0.      -0.      18.93151]
v2 [ 4.1966 -0.      0.    ]
v3 [-0.       4.19659  0.     ]
d2 [-0.     -4.1966  0.    ]
d3 [ 4.19659  0.      -0.     ]
Identity
invC4_v1
C2_v1
M_v1C4_v1
M_v2
invM_v3C4_v1
M_v3
invM_v2C4_v1
CrI2  P-3m1 (164) [-3m]
v1 [ 0.       0.      18.22368]
v2 [4.11569 0.      0.     ]
v3 [-2.05785  3.56429  0.     ]
d2 [ 0.      -4.11569  0.     ]
d3 [ 3.56429  2.05785 -0.     ]
Identity
Inversion
C3_v1
invC3_v1
invM_v1C6_v1
M_v1C6_v1
invM_d3C6_v1
M_d3C6_v1
C2_v2
M_v2
C2_v3
M_v3


In [3]:
print( c2t.sym_label(Rs[0],cell) );


Identity


In [4]:
lat_vec = [[5.3422,0.0,0.0],[0.0,5.3422,0.0],[0.0,0.0,1.9515 *5.3422 ]];
numbers = [1,2,2];
positions= np.array([ (0, 0,0.5) , (0.5,0.5,0.257386261), (0.5,0.5,0.742613739 )]).dot(lat_vec)
cell= ( np.array(lat_vec), np.array(positions),np.array(numbers)) ;

symmetry= c2t.get_symmetry(cell);
spg = symmetry["international"]; spgn = symmetry["number"];
ptg = symmetry["pointgroup"];
print( " {} ({}) [{}]".format(spg,spgn,ptg ))                
sym_ops= c2t.get_symmetry_matrices(cell);
Rs, ts = sym_ops['rotations'] , sym_ops['translations'];
for v, k in c2t.symmetry_directions(cell).items():
    print(v,k)
for R in Rs:
    print( c2t.sym_label(R,cell) );


 Amm2 (38) [mm2]
v1 [ 0.         0.        10.4253033]
v2 [5.3422 0.     0.    ]
v3 [0.     5.3422 0.    ]
d2 [ 0.     -5.3422  0.    ]
d3 [5.3422 0.     0.    ]
Identity
invM_v2C4_v1
M_v1
M_v3C4_v1


In [5]:
print(( np.array(lat_vec), np.array(positions),np.array(numbers)) )

(array([[ 5.3422   ,  0.       ,  0.       ],
       [ 0.       ,  5.3422   ,  0.       ],
       [ 0.       ,  0.       , 10.4253033]]), array([[0.        , 0.        , 5.21265165],
       [2.6711    , 2.6711    , 2.68332984],
       [2.6711    , 2.6711    , 7.74197346]]), array([1, 2, 2]))


In [6]:
cell

(array([[ 5.3422   ,  0.       ,  0.       ],
        [ 0.       ,  5.3422   ,  0.       ],
        [ 0.       ,  0.       , 10.4253033]]),
 array([[0.        , 0.        , 5.21265165],
        [2.6711    , 2.6711    , 2.68332984],
        [2.6711    , 2.6711    , 7.74197346]]),
 array([1, 2, 2]))